# Meiro - assignment solution

Hello there!
Thanks a lot for being a part of the interview for data position in Meiro, I am glad you got in
touch and made it so far. As a last task I have a data file for you I would like you to analyze.
This practice should not take you more than 2 hours. In the attachment you can find a
compressed file containing newline delimited JSON objects.
Please, process the file and based on the contents prepare simple data model for a relational
database suitable for asking analytical questions. You can use any database you like - from
SQLite, MySQL, PostgreSQL, MS Access, MS SQL Server to Redshift, BigQuery, Snowflake
and more.
Then answer following two questions:
1) Which user spent the most money on products on all Fridays?
2) What are the best 3 products in each location of a user based on quantity?
Please, document the whole process of your work and thinking (exploring the file, unzipping it,
processing, loading to the DB of your choice, writing SQL queries, ...) - it can have the form of
Jupyter Notebook, RStudio file, PDF, MS Word or Google Docs document, blogpost - whatever
works for you.
Should you have any questions, please, do not hesitate to let me know.
Thanks a lot and have a great day1

**For this solution I'm trying different approach using pyjsonq - librabry taht help to query data directly in json**

In [80]:
# Import libraries I'll use during session
from pyjsonq import JsonQ
from datetime import datetime
import json

In [52]:
# Function to convert ndjson file to json (pyjsonq can't access ndjson file)
def ndjson_to_json(ndjson_file, json_file):
    with open(ndjson_file, 'r') as ndjson_file:
        ndjson_data = [json.loads(line) for line in ndjson_file]
    with open(json_file, 'w') as json_file:
        json.dump(ndjson_data, json_file)
ndjson_to_json('data/data.ndjson', 'data/data.json')

In [51]:
# Read data
json_data = JsonQ('data/data.json')

In [11]:
json_raw = json_data._raw_data

## First Question

In [76]:
# Initialize dictionary, match only Fridays via timestamp, access data, sort data from descending, print all spendings and max spending.
total_spending = {}
for transaction in json_raw:
    transaction_date = datetime.fromtimestamp(transaction['created'])
    if transaction_date.weekday() == 4:
        user_id = transaction['user']['id']
        spending = sum(product['price'] for product in transaction['products'])
        if user_id in total_spending:
            total_spending[user_id] +=spending
        else:
            total_spending[user_id] = spending
sorted_spending = sorted(total_spending.items(), key=lambda x: x[1], reverse=True)
if total_spending:
    max_spending_user = sorted_spending[0][0]
    print("User with ID:", max_spending_user, "spent the most money on products on all Fridays.")
print("All users spendnings for Fridays")
for user_id, spending in sorted_spending:
    print("User with ID:", user_id, "spent amount:", spending)

User with ID: 4 spent the most money on products on all Fridays.
All users spendnings for Fridays
User with ID: 4 spent amount: 869810
User with ID: 1 spent amount: 862980
User with ID: 5 spent amount: 855470
User with ID: 2 spent amount: 835880
User with ID: 3 spent amount: 831690
User with ID: 7 spent amount: 825720
User with ID: 6 spent amount: 825350
User with ID: 0 spent amount: 814700


### User with ID: 4 spent the most money on products on all Fridays with spent amount: 869810

## Second Question

In [102]:
# Initialize dictionary, for loop for location, loop for product id and name, initialize top_3_product dictionary, loop for quanity amount and print data
location_product_count = {}
for transaction in json_raw:
    location = transaction['user']['city']
    if location not in location_product_count:
        location_product_count[location] = {}
    for product in transaction['products']:
        product_id = product['id']
        product_name = product['name']
        if product_id in location_product_count[location]:
            location_product_count[location][product_id][product_name] +=1
        else:
            location_product_count[location][product_id] = {product_name: 1}
top_3_products = {}
for location, product_count in location_product_count.items():
        top_products = sorted(product_count.items(), key=lambda x: list(x[1].values())[0], reverse = True)[:3]
        top_products_names = [(product_id, list(count.keys())[0], list(count.values())[0]) for product_id, count in top_products]
        top_3_products[location] = top_products_names

### Top 3 products for each location are displayed in table below

In [103]:
print("Top 3 products by quantity for each city")
for location, top_products in top_3_products.items():
    print("City:", location)
    for product_id, product_name, count in top_products:
        print("Product ID", product_id, "- product:", product_name, "with quantity of", count)

Top 3 products by quantity for each city
City: Sydney
Product ID 12 - product: Product M with quantity of 4335
Product ID 10 - product: Product K with quantity of 4318
Product ID 11 - product: Product L with quantity of 4313
City: Prague
Product ID 2 - product: Product C with quantity of 4400
Product ID 11 - product: Product L with quantity of 4388
Product ID 6 - product: Product G with quantity of 4362
City: Singapore
Product ID 15 - product: Product P with quantity of 4479
Product ID 13 - product: Product N with quantity of 4396
Product ID 2 - product: Product C with quantity of 4381
City: Jakarta
Product ID 3 - product: Product D with quantity of 4328
Product ID 9 - product: Product J with quantity of 4309
Product ID 2 - product: Product C with quantity of 4305
City: Melbourne
Product ID 9 - product: Product J with quantity of 4438
Product ID 3 - product: Product D with quantity of 4400
Product ID 8 - product: Product I with quantity of 4391
City: Kuala Lumpur
Product ID 1 - product